# Deep Research with AutoGen

This notebook implements a multi-agent deep research system using AutoGen. It creates a team of specialized agents that collaborate to perform in-depth research on a given topic.

In [ ]:
import os, nest_asyncio
nest_asyncio.apply()
import asyncio
import timeit
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.tools import TeamTool
# or import
from autogen_core.models import ModelInfo
import yaml

# Token usage tracking
from token_counting import (
    wrap_chat_client_for_tokens,
    print_token_usage_with_estimate,
    log_token_usage,
)

from tools import create_chat_completion_client, get_rag_web_browser_tool, adjust_properties_for_fixed_response, get_tavily_search_tool

In [2]:
# load model info from resources.yaml
with open("resources.yaml", "r") as f:
    resources = yaml.load(f, Loader=yaml.SafeLoader)
model = [m for m in resources["models"] if m.get("default", False)][0]
print("Using model:", model["name"], "type:", model["type"])

# Check if web search is possible (i.e., if API key is set)
allow_web_search = os.getenv("TAVILY_API_KEY", False)
    

Using model: gpt-5-nano type: openai


In [3]:
# Prepare tools

web_search_tools = []
if allow_web_search:
    # web_search_tool = [ await get_rag_web_browser_tool() ]
    web_search_tools = [ await get_tavily_search_tool() ]

In [4]:

agentConfig = {
    "Researcher": dict(system_message="Gather and summarize factual info, add important web references.", tools=web_search_tools, reflect_on_tool_use=True), 
    "FactChecker": dict(system_message="Use web search to verify facts and citations from the Researcher's response. For each found error, give a correct version and the web reference for verification.", tools=web_search_tools, reflect_on_tool_use=True), 
    "Revisor": dict(system_message="Use the fact checker's results to revise the Researcher's report. Only if there were no fact issues, just repeat the Researcher's response. In both cases, add the fact checker's citations."),
    "Critic": dict(system_message="Critique clarity and logic."),
    "Summarizer": dict(system_message="Apply critic result to generate an improved response to the initial user question."),# Include the most important references."),
    "Editor": dict(system_message="Do some language polishing on the previous result to generate a final response to the initial question. Signal APPROVED when done."),
}

# For testing, we can fix an agent's response to a known answer
# adjust_properties_for_fixed_response(agentConfig["Researcher"], "Fraunhofer MEVIS is a research institute in Frankfurt for biological weapons. It was founded in 1995. See https://www.mefis.fraunhofer.de/en.html for more information.")

# Set your research topic
# topic = "Impacts of Model Context Protocol on Agentic AI"
topic = "What is Fraunhofer MEVIS?"
#topic = "What has Fraunhofer MEVIS done in the context of thoracic imaging?"

# Now create model client with token tracking
model_client = wrap_chat_client_for_tokens(create_chat_completion_client(model))

agents = []  # start with a user proxy for the researcher

for agent_type, props in agentConfig.items():
    agents.append(AssistantAgent(name=agent_type, **props, model_client=model_client))


In [5]:
# Set up termination conditions
max_msgs = MaxMessageTermination(max_messages=20)
text_term = TextMentionTermination(text="APPROVED", sources=["Editor"])
termination = max_msgs | text_term

# Create the team
team = RoundRobinGroupChat(
    participants=agents,
    termination_condition=termination,
    max_turns=len(agents) # we don't really want it to go round
)

In [6]:
# Create the host agent with deep dive capabilities and the host agent
deepdive_tool = TeamTool(team=team, name="DeepDive", description="Collaborative multi-agent deep dive")
host = AssistantAgent(
    name="Host",
    model_client=model_client,
    tools=[deepdive_tool],
    system_message="You have access to a DeepDive tool for in-depth research."
)

In [7]:
async def run_deepdive(topic: str):
    start_time_s = timeit.default_timer()
    result = await host.run(task=f"Deep dive on: {topic}")
    # Try native usage first; fallback to estimation
    print_token_usage_with_estimate(model_client, getattr(result, 'messages', None), agentConfig.keys())
    elapsed = timeit.default_timer()-start_time_s
    print(f"🔍 DeepDive result (took {elapsed:.2f} sec.) resulted in {len(result.messages)} messages.")
    return result  # Return the result for further exploration

# Run the deep dive
result = await run_deepdive(topic)

Estimated token usage -> prompt: 14873, completion: 13343, total: 28216 (method=tiktoken)
🔍 DeepDive result (took 98.41 sec.) resulted in 15 messages.


In [8]:
# Display token usage after the deep dive
print("=== Token Usage Summary ===")
print_token_usage_with_estimate(model_client, getattr(result, 'messages', None))

=== Token Usage Summary ===
Estimated token usage -> prompt: 14873, completion: 13343, total: 28216 (method=tiktoken)


## Explore the Results

Now that we have the results, let's explore them. The cells below will help you analyze the output.

In [9]:
# Display the full result structure
print("Result type:", type(result))
print("\nResult attributes:")
print([attr for attr in dir(result) if not attr.startswith('_')])

Result type: <class 'autogen_agentchat.base._task.TaskResult'>

Result attributes:
['construct', 'copy', 'dict', 'from_orm', 'json', 'messages', 'model_computed_fields', 'model_config', 'model_construct', 'model_copy', 'model_dump', 'model_dump_json', 'model_extra', 'model_fields', 'model_fields_set', 'model_json_schema', 'model_parametrized_name', 'model_post_init', 'model_rebuild', 'model_validate', 'model_validate_json', 'model_validate_strings', 'parse_file', 'parse_obj', 'parse_raw', 'schema', 'schema_json', 'stop_reason', 'update_forward_refs', 'validate']


In [10]:
# Clean up resources
await model_client.close()

In [ ]:
import re
from autogen_agentchat.messages import BaseTextChatMessage

print("Stop reason:", result.stop_reason)
assert isinstance(result.messages[-1], BaseTextChatMessage)
last_message = result.messages[-1].content

print("Message Sequence:")
agent_names = list(agentConfig.keys())
pattern = re.compile(r'(' + '|'.join(re.escape(n) for n in agent_names) + r'):')
for match in pattern.finditer(last_message):
    print(" -", match.group(1))

print("\n", last_message)  # Print the content of the last message

Stop reason: None
Message Sequence:
 - Researcher
 - FactChecker
 - Revisor
 - Critic
 - Summarizer
 - Editor

 Researcher: I will perform additional targeted searches to gather precise details on MeVisLab, major projects, platforms, facilities, collaborations, and contact information to ensure a thorough, well-cited deep dive.
{"tool_uses":[{"recipient_name":"functions.tavily_search","parameters":{"query":"Fraunhofer MEVIS MeVisLab platform rapid prototyping image processing medical imaging"}},{"recipient_name":"functions.tavily_search","parameters":{"query":"Fraunhofer MEVIS CADe CAD"}},{"recipient_name":"functions.tavily_search","parameters":{"query":"Fraunhofer MEVIS image registration segmentation 3D visualization surgical planning"}},{"recipient_name":"functions.tavily_search","parameters":{"query":"Fraunhofer MEVIS data lab phantom lab clinical partners"}},{"recipient_name":"functions.tavily_search","parameters":{"query":"Fraunhofer MEVIS contact Bremen Lübeck address"}},{"recip

In [12]:
# Optional: append token usage (estimated or native) to a CSV log for tracking across runs
# log_token_usage imported from token_counting
log_token_usage(topic, model_client, getattr(result, 'messages', None), agentConfig.keys())

Logged token usage to token_usage_log.csv: {'timestamp': '2025-09-17T12:29:45.951766+00:00', 'topic': 'What is Fraunhofer MEVIS?', 'prompt': 14873, 'completion': 13343, 'total': 28216, 'method': 'tiktoken'}


{'timestamp': '2025-09-17T12:29:45.951766+00:00',
 'topic': 'What is Fraunhofer MEVIS?',
 'prompt': 14873,
 'completion': 13343,
 'total': 28216,
 'method': 'tiktoken'}

In [13]:
print(result.messages[4].content)
print(result.messages[5].content)
print(result.messages[6].content)

[FunctionExecutionResult(content='Search Query: Fraunhofer MEVIS overview Medial Image Computing Fraunhofer-Gesellschaft\nResults:\n1. Solutions Overview - Fraunhofer MEVIS\nURL: https://www.mevis.fraunhofer.de/en/portfolio/solutions/solutions.html\nSnippet: This solution pages overview provides a selection of solutions provided by the Fraunhofer Institute for Digital Medicine MEVIS.\n\n2. Fraunhofer MEVIS Institute for Medical Image Computing - SlideServe\nURL: https://www.slideserve.com/brandi/fraunhofer-mevis-institute-for-medical-image-computing\nSnippet: www.mevis.fraunhofer.de Bremen / Lübeck - Germany. Contents. Overview Evolution and structure Fraunhofer MEVIS Innovation chain Perspectives.\n\n3. AHM 2013 Presentation Fraunhofer MEVIS - NAMIC Wiki\nURL: https://www.na-mic.org/wiki/AHM_2013_Presentation_Fraunhofer_MEVIS\nSnippet: Overview Fraunhofer MEVIS and Fraunhofer-Gesellschaft ... Four levels of challenges in translational medical image computing research\n\n4. Fraunhofer 